### Create power tables

In [298]:
import pandas as pd
P1P2 = pd.read_csv('data/Effekt FV P1 och P2', sep=',')[['ts', 'Effekt FV P1 och P2']]
P5 = pd.read_csv('data/Effekt FV P5', sep=',')[['ts', 'Effekt FV P5']]
P6 = pd.read_csv('data/Effekt FV P6', sep=',')[['ts', 'Effekt FV P6']]

P1P2['ts']= pd.to_datetime(P1P2['ts'])
P5['ts']= pd.to_datetime(P5['ts'])
P6['ts']= pd.to_datetime(P6['ts'])

data = P1P2.merge(P5, on='ts').merge(P6, on='ts')

## Create temperature tables

In [300]:
temp_Hallstahammar = pd.read_csv('data/T Ute  Hallstahammar', sep=',')[['ts', 'T Ute  Hallstahammar']]
temp_KVV = pd.read_csv('data/T Ute KVV', sep=',')[['ts', 'T Ute KVV']]
temp_Knuthagen = pd.read_csv('data/T Ute Knuthagen', sep=',')[['ts', 'T Ute Knuthagen']]
temp_Ronnby = pd.read_csv('data/T Ute RĒnnby', sep=',')[['ts', 'T Ute Rönnby']]
temp_Skultuna = pd.read_csv('data/T Ute Skultuna', sep=',')[['ts', 'T Ute Skultuna']]
temp_prediction = pd.read_csv('data/Temp prognos', sep=',')[['ts', 'Temp prognos']]
temp_recommendation = pd.read_csv('data/Temp rekommenderad fram FV', sep=',')[['ts', 'Temp rekommenderad fram FV']]
temp_in = pd.read_csv('data/Temp medel åter FV', sep=',')[['ts', 'Temp medel åter FV']]
temp_out = pd.read_csv('data/Temp vĄgd fram FV', sep=',')[['ts', 'Temp vägd fram FV']]

temp_Hallstahammar['ts']= pd.to_datetime(temp_Hallstahammar['ts'])
temp_KVV['ts']= pd.to_datetime(temp_KVV['ts'])
temp_Knuthagen['ts']= pd.to_datetime(temp_Knuthagen['ts'])
temp_Ronnby['ts']= pd.to_datetime(temp_Ronnby['ts'])
temp_Skultuna['ts']= pd.to_datetime(temp_Skultuna['ts'])
temp_prediction['ts']= pd.to_datetime(temp_prediction['ts'])
temp_recommendation['ts']= pd.to_datetime(temp_recommendation['ts'])
temp_in['ts']= pd.to_datetime(temp_in['ts'])
temp_out['ts']= pd.to_datetime(temp_out['ts'])

## Merge data to one data frame

In [301]:
data = data.merge(temp_Hallstahammar, on='ts')
data = data.merge(temp_KVV, on='ts')
data = data.merge(temp_Knuthagen, on='ts')
data = data.merge(temp_Ronnby, on='ts')
data = data.merge(temp_Skultuna, on='ts')
data = data.merge(temp_prediction, on='ts')
data = data.merge(temp_recommendation, on='ts')
data = data.merge(temp_in, on='ts')
data = data.merge(temp_out, on='ts')

data = data.rename(columns=
                   {"T Ute  Hallstahammar": "temp_Hallstahammar", 
                    "T Ute KVV": "temp_KVV",
                    "T Ute Knuthagen": "temp_Knuthagen",
                    "T Ute Rönnby": "temp_Ronnby",
                    "T Ute Skultuna": "temp_Skultuna",
                    "Temp prognos": "temp_prediction",
                    "Temp rekommenderad fram FV": "temp_recommendation",
                    "Temp medel åter FV": "temp_in",
                    "Temp vägd fram FV": "temp_out",
                    "Effekt FV P1 och P2": "power_p1_p2",
                    "Effekt FV P5": "power_p5",
                    "Effekt FV P6": "power_p6"
                   })

## Remove invalid records and split the data frame at gap borders

In [302]:
print (len(data))

13829


In [303]:
data = data[data['temp_in'] != 0.0]
print (len(data))

13104


In [304]:
from datetime import timedelta
data['gap'] = (data['ts'] - data['ts'].shift(1)) != timedelta(hours=1)

In [305]:
dflist = []
attributes = ['power_p1_p2', 'power_p5', 'power_p6', 'temp_Hallstahammar', 'temp_KVV', 'temp_Knuthagen', 
              'temp_Ronnby', 'temp_Skultuna', 'temp_in', 'temp_out']
start = 0
for stop in range(1, len(data)):
    if data.iloc[stop]['gap']:
        dflist.append(data[start:stop][attributes])
        start = stop
len(dflist)

23

## Create time series data

In [306]:
w = 5 #window size
s = 1 #step
X_all = []
y_all = []

In [307]:
for df in dflist:
    for i in range(0, len(df)-w-1, s):
        X_all.append(df[i:i+w].values)
        y_all.append(df.iloc[i+w]['temp_in'])

In [308]:
X_all = np.array(X_all)
y_all = np.array(y_all)
X_all_flat = X_all.reshape(X_all.shape[0], -1)
y_all_baseline = X_all_flat[:, -2]

print (X_all.shape)
print (X_all_flat.shape)
print (y_all.shape)
print (y_all_baseline.shape)

(12976, 5, 10)
(12976, 50)
(12976,)
(12976,)


## Train a GBR model for various train/test splits and compute evaluation statistics

In [309]:
def rmse(predictions, true_values):
    return np.sqrt(((predictions - true_values) ** 2).mean())

In [310]:
import random
from sklearn.ensemble import GradientBoostingRegressor

mean_baseline = []
last_values_baseline = []
ml_model = []
iters = 20
train_size = int(0.9*len(X_all))

for i in range(iters):
    all_data = list(zip(X_all_flat, y_all, y_all_baseline))
    random.shuffle(all_data)
    X_all_flat_rand, y_all_rand, y_all_rand_baseline = zip(*all_data)
    X_train = np.array(X_all_flat_rand[:train_size])
    y_train = np.array(y_all_rand[:train_size])
    X_test = np.array(X_all_flat_rand[train_size:])
    y_test = np.array(y_all_rand[train_size:])
    y_baseline = np.array(y_all_rand_baseline[train_size:])
    
    print ("Training iteration {}.".format(i+1))
    regr = GradientBoostingRegressor(n_estimators=500)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    
    m = np.mean(y_train)
    y_mean = np.array([m for i in range(len(y_test))])
    
    mean_baseline.append(rmse(y_mean, y_test))
    last_values_baseline.append(rmse(y_baseline, y_test))
    ml_model.append(rmse(y_pred, y_test))

Training iteration 1.
Training iteration 2.
Training iteration 3.
Training iteration 4.
Training iteration 5.
Training iteration 6.
Training iteration 7.
Training iteration 8.
Training iteration 9.
Training iteration 10.
Training iteration 11.
Training iteration 12.
Training iteration 13.
Training iteration 14.
Training iteration 15.
Training iteration 16.
Training iteration 17.
Training iteration 18.
Training iteration 19.
Training iteration 20.
[3.8412267473681285, 3.5845522230930356, 3.7505293645288242, 3.7416450267097567, 3.933294716165963, 3.6878937407594083, 3.8412008181113015, 3.6344971690934833, 3.9001544173033618, 3.706636560762584, 3.717979201134607, 3.8794077901574178, 3.707628370871983, 4.041147009494571, 3.932528354810767, 3.898676768357415, 3.7373850600751943, 3.7022200927828752, 3.790211670239497, 3.80058121458492]
[0.8267962864831379, 0.7846968222476624, 0.8830378669515391, 0.6952048152894295, 0.7292106288805978, 0.8418961107269054, 0.7167960297203496, 0.815264282541273

In [311]:
print ("Mean baseline: {} +- {}.".format(np.mean(mean_baseline), np.std(mean_baseline)))
print ("Last values baseline: {} +- {}.".format(np.mean(last_values_baseline), np.std(last_values_baseline)))
print ("ML model: {} +- {}.".format(np.mean(ml_model), np.std(ml_model)))

Mean baseline: 3.7914698158202556 +- 0.1120897742040861.
Last values baseline: 0.8470553471994166 +- 0.23414835852342136.
ML model: 0.8266906751246712 +- 0.20548856310809796.


## Recurrent Neural Network Model

The quality of this model is poor in comparison to simple GBR. In needs more elaboration :) 

In [316]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.1)
scaler = StandardScaler()

orig_shape = X_train.shape
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(orig_shape)
orig_shape = X_test.shape
X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(orig_shape)

In [318]:
#%tensorflow_version 2.x
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, LSTM, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam

model = Sequential()
model.add(LSTM(12, input_shape=(w, 10), dropout=0.4, return_sequences=True))
model.add(LSTM(12, dropout=0.4, return_sequences=False))
model.add(Dense(1))
model.add(Activation('linear'))

#optimizer = RMSprop(lr=0.001)
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)

In [319]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=8, epochs=100)

Train on 11678 samples, validate on 1298 samples
Epoch 1/100
11678/11678 [==============================] - 16s 1ms/sample - loss: 1177.9274 - val_loss: 631.8058
Epoch 2/100
11678/11678 [==============================] - 13s 1ms/sample - loss: 332.3444 - val_loss: 151.0508
Epoch 3/100
11678/11678 [==============================] - 13s 1ms/sample - loss: 64.0223 - val_loss: 26.4714
Epoch 4/100
11678/11678 [==============================] - 13s 1ms/sample - loss: 16.2977 - val_loss: 14.8127
Epoch 5/100
11678/11678 [==============================] - 13s 1ms/sample - loss: 14.1913 - val_loss: 14.4803
Epoch 6/100
11678/11678 [==============================] - 13s 1ms/sample - loss: 9.1467 - val_loss: 5.3951
Epoch 7/100
11678/11678 [==============================] - 13s 1ms/sample - loss: 5.3418 - val_loss: 3.5799
Epoch 8/100
11678/11678 [==============================] - 13s 1ms/sample - loss: 4.4004 - val_loss: 2.9089
Epoch 9/100
11678/11678 [==============================] - 13s 1ms/sampl